In [1]:
import numpy as np
import pandas as pd
import spacy
import en_core_web_sm
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
import pickle
from IPython.display import clear_output

# ------- FUNÇÕES (brevemente comentadas)

# todo o pré processamento, com exceção da vetorização
def text_preproc(dataframe, series_name, target, pos_label):
    # remoção de stop words
    dataframe[series_name] = dataframe[series_name].apply(lambda x:
                                                          [token for token in nlp(x) if not token.is_stop])
    # remoção de pontuação
    dataframe[series_name] = dataframe[series_name].apply(lambda x:
                                                          [token for token in x if not token.is_punct])
    # lemmatização
    dataframe[series_name] = dataframe[series_name].apply(lambda x:
                                                          [token.lemma_ for token in x])
    # passando tokens para formato minúsculo
    dataframe[series_name] = dataframe[series_name].apply(lambda x:
                                                          [token.lower() for token in x])
    # transformando listas de tokens em strings (para vetorização)
    dataframe[series_name] = dataframe[series_name].apply(lambda x: ' '.join(x))
    # binarizando alvo
    dataframe[target] = dataframe[target].apply(lambda x: int(x == pos_label))

# obtenção da acurácia, com validação cruzada, sem otimização
def simple_acc(mdl, split_dicti):
    
    accuracy_list = []
    accuracy_list_overfit = []
    
    for key in split_dicti.keys(): # para cada fold (cada um é armazenado em dicionário)

        data = split_dicti[key] # acessando key cujo valor é um dicionário, contendo 
                                # X_train, X_test, y_train, y_test

        mdl.fit(data['X_train'], data['y_train'])

        y_pred = mdl.predict(data['X_test'])  
        y_pred_train = mdl.predict(data['X_train']) # para obtenção da métrica para porção de treino, 
                                                    # a fim de calcular o overfit

        accuracy_list_overfit.append(accuracy_score(y_true = data['y_train'], y_pred = y_pred_train))
        accuracy_list.append(accuracy_score(y_true = data['y_test'], y_pred = y_pred))
    
    # este 'retorno' é interpretado pela próxima função
    return score_mean_std_overfit(accuracy_list, accuracy_list_overfit)

# função para obter média, desvio padrão e overfit para diferentes métricas ('chamada' pela anterior)
def score_mean_std_overfit(test, train):
    return {'test_avg':np.mean(test), 'test_std':np.std(test), 
            'overfit_avg':np.mean([i[0] - i[1] for i in zip(train, test)])}

# obtenção de splits de treino e teste (vetorizando-os)
def get_folds(x, y, folds, seed):
    # criando lista de listas com indexes de treino e teste, de acordo com KFold
    kf = KFold(n_splits = folds, shuffle = True, random_state = seed)
    split_index_list = [[train_index.tolist(), test_index.tolist()]\
                        for train_index, test_index in kf.split(x, y)]
    # vetorizando cada split de treino e teste
    count = 0
    dicti = {}
    for n in range(len(split_index_list)): # <- percorre lista de listas (contendo índices de treino e 
                                                                          # teste para para cada fold)
        train_indexes = split_index_list[n][0] # variável contendo índices de treino
        test_indexes = split_index_list[n][1] # variável contendo índices de teste

        X_train = x.iloc[train_indexes] # declarando X_train, X_test, y_train, y_test
        X_test = x.iloc[test_indexes]
        y_train = y.iloc[train_indexes]
        y_test = y.iloc[test_indexes]

        vect = TfidfVectorizer()
        X_train = vect.fit_transform(X_train) # ajuste aos mesmos e transformação dos dados de treino
        X_test = vect.transform(X_test)       # dados de teste com este vetorizador ajustado ao treino

        dicti[str(count)] = {'X_train':X_train, 'X_test':X_test, 'y_train':y_train, 'y_test':y_test}
        count += 1
        
    return dicti

# obtenção de métricas para combinações de hiperparâmetros (para um 'par' de treino e teste)
def single_fold_tuning(partition, combo_dict):
    
    lista = []
    for key in combo_dict:  
        # verbosidade
        clear_output()
        print(key, '/', len(combo_dict) - 1)

        combo = combo_dict[key]['params']
        c = combo['C']
        g = combo['gamma']
        k = combo['kernel']

        svc = SVC(C = c, gamma = g, kernel = k)
        svc.fit(partition['X_train'], partition['y_train'])

        lista.append([str(str(c) + ',' + str(g) + ',' + str(k)), accuracy_score(y_true = partition['y_test'],
                                              y_pred = svc.predict(partition['X_test']))])
    
    return lista

df = pd.read_csv('pycoders_reviews.tsv', sep = '\t')
nlp = en_core_web_sm.load()
seed = 0

In [2]:
# em virtude do tempo de duração de alguns dos processos envolvidos neste projeto, a maior parte destes 
# teve seus resultados armazenados em arquivos '.pkl', todo o código envolvido está escrito no notebook, 
# como comentário, todavia, apenas as linhas que carregam tais arquivos estão escritas como código

# neste projeto, consideramos os seguintes modelos: 'MultinomialNB', 'LogisticRegression', 'SVC'; por quê ?
# referências:
# https://monkeylearn.com/blog/sentiment-analysis-machine-learning/#:~:text=Naive%20Bayes%20is%20a%20fairly,be%20considered%20positive%20or%20negative.&text=Basically%2C%20Naive%20Bayes%20calculates%20words%20against%20each%20other.
# https://theappsolutions.com/blog/development/sentiment-analysis/

In [3]:
# balanceamento no dataframe original
df['label'].value_counts(1)

neg    0.509891
pos    0.490109
Name: label, dtype: float64

In [4]:
# verificando consistência dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 2 columns):
label     9200 non-null object
review    9200 non-null object
dtypes: object(2)
memory usage: 143.9+ KB


In [5]:
# posteriormente, será feita validação cruzada(random_state = seed), nesta célula, 'reproduzo' os splits,
# analisando o balanceamento de cada um
kf = KFold(n_splits = 3, shuffle = True, random_state = seed)
split_index_list = [[train_index.tolist(), test_index.tolist()]\
                    for train_index, test_index in kf.split(df['review'], df['label'])]

for n in range(len(split_index_list)):                                                                     
    print(df['label'].iloc[split_index_list[n][0]].value_counts(1))
    print(df['label'].iloc[split_index_list[n][1]].value_counts(1))
    print('')

neg    0.503995
pos    0.496005
Name: label, dtype: float64
neg    0.521682
pos    0.478318
Name: label, dtype: float64

neg    0.52242
pos    0.47758
Name: label, dtype: float64
pos    0.515161
neg    0.484839
Name: label, dtype: float64

neg    0.503261
pos    0.496739
Name: label, dtype: float64
neg    0.523157
pos    0.476843
Name: label, dtype: float64



In [6]:
# pré processando base inteira(operações 'independentes', como remoção de stop words, lemmatização ...)
# text_preproc(df, 'review', 'label', 'pos')
# pickle.dump(df, open('full_pre_proc_df.pkl', 'wb'))

# validação cruzada com 3 folds, as variáveis explicativas são transformadas com vetorizadores separados
# cada vetorizador é ajustado aos dados de treino, depois este transforma dados de treino e teste,
# tinha a intenção de usar 5 folds, mas a otimização do SVC é muito lenta(na minha opinião)

# como planejo otimizar hiperparâmetros de um dos modelos, além da necessidade de vetorizar cada
# split de teste com objeto ajustado ao split de treino, opto por implementar a validação cruzada de uma
# forma diferente(que me permitisse entender melhor o que acontece); a função get_folds() retorna um dicionário,
# este que possuirá uma key para cada fold solicitado, cada uma destas keys possuirá como valor, um dicionário
# com keys 'X_train', 'X_test', 'y_train', 'y_test', implementando este processo, faço a vetorização de treino
# e teste com vetorizador ajustado ao split de treino, os splits retornados estão completamente pré processados
# esta organização permite também realizar a otimização por fold individual, para que não precisasse ficar 
# com a máquina rodando ininterruptamente desde o início até o fim da mesma

# split_dict = get_folds(df['review'], df['label'], 3, seed)
# pickle.dump(split_dict, open('full_split_dictionary.pkl', 'wb'))

df = pickle.load(open('full_pre_proc_df.pkl', 'rb'))
split_dict = pickle.load(open('full_split_dictionary.pkl', 'rb'))

In [7]:
# obtendo acurácia de regressão logística e naive bayes (validação cruzada de 3 folds), os dicionários
# retornados serão utilizados para a confecção de um dataframe que consolidará informações sobre as 
# performances de todos os modelos testados

# lr_report = simple_acc(LogisticRegression(), split_dict)
# lr_report['model'] = 'LogisticRegression'

# mnb_report = simple_acc(MultinomialNB(), split_dict)
# mnb_report['model'] = 'MultinomialNB'

# pickle.dump(lr_report, open('lr_report.pkl', 'wb'))
# pickle.dump(mnb_report, open('mnb_report.pkl', 'wb'))

lr_report = pickle.load(open('lr_report.pkl', 'rb'))
mnb_report = pickle.load(open('mnb_report.pkl', 'rb'))

In [8]:
# criando dicionário contendo as possíveis combinações de hiperparâmetros do SVC (otimização)

combo_dict = {}
combo_count = 0

for c in [0.1, 1, 10, 100]:                     # valores testados para parâmetro: C
    for g in [1, 0.1, 0.01, 0.001]:             #                //              : gamma
        for k in ['rbf', 'poly', 'sigmoid']:    #                //              : kernel
            
            combo_dict[combo_count] = {'params':{'C':c, 'gamma':g, 'kernel':k}}
            combo_count += 1
            
# será obtida a métrica de cada combinação para cada fold (para que possamos realizar a otimização em momentos
# distintos), por exemplo, no caso de 3 folds, podemos obter a métrica de cada uma das 48 combinações de 
# hiperparâmteros para o primeiro fold, agora estamos diante de um dicionário contendo um valor para cada 
# combinação, em seguida, repetimos o processo com o segundo fold, agora, dois dicionários, considerando estes, 
# temos duas métricas para cada combinação, obtemos métricas para cada combinação em cada fold (número 
# arbitrário) finalmente, inserimos todas as combinações e scores em um dataframe, para 3 folds, este 
# terá, para cada possível combinação de hiperparâmetros, 3 métricas, assim sendo, ao realizar um groupby,
# obtemos a média das métricas para determinada combinação, entre os 3 folds, em seguida, buscamos pelo
# valor de score mais alto e consultamos a combinação que nos levou a este, finalizando a otimização

# não rodar 'single_fold_tuning()' para vários folds em uma mesma célula (verbosidade, além de derrotar o 
# propósito; o motivo da otimização ter sido realizada desta forma foi (falta de paciência)
# eu não querer separar x período de tempo contínuo do dia para deixar o computador trabalhando, então dividi
# em 3 * (x/3))

In [9]:
# fold1_list = single_fold_tuning(split_dict['0'], combo_dict)

In [10]:
# pickle.dump(fold1_list, open('fold1_list.pkl', 'wb'))
fold1_list = pickle.load(open('fold1_list.pkl', 'rb'))

In [11]:
# fold2_list = single_fold_tuning(split_dict['1'], combo_dict)

In [12]:
# pickle.dump(fold2_list, open('fold2_list.pkl', 'wb'))
fold2_list = pickle.load(open('fold2_list.pkl', 'rb'))

In [13]:
# fold3_list = single_fold_tuning(split_dict['2'], combo_dict)

In [14]:
# pickle.dump(fold3_list, open('fold3_list.pkl', 'wb'))
fold3_list = pickle.load(open('fold3_list.pkl', 'rb'))

In [15]:
tuning_data = fold1_list + fold2_list + fold3_list
tuning_df = pd.DataFrame(tuning_data, columns = ['params', 'score'])
tuning_df.groupby('params').mean().sort_values('score', ascending = False).head(1)

,score
params,
"1,1,rbf",0.848043


In [16]:
tuning_df.loc[tuning_df.params == '1,1,rbf']

,params,score
12,"1,1,rbf",0.852951
60,"1,1,rbf",0.848060
108,"1,1,rbf",0.843118


In [17]:
# concluímos que o conjunto de hiperparâmetros ideal para o SVC é C = 1, gamma = 1, kernel = 'rbf'
# prosseguimos para a obtenção dos dados referentes á perfomance de um modelo com estes hiperparâmetros

# svc_report = simple_acc(SVC(C = 1, gamma = 1, kernel = 'rbf'), split_dict)
# svc_report['model'] = 'SVC'

In [18]:
# pickle.dump(svc_report, open('svc_report.pkl', 'wb'))
svc_report = pickle.load(open('svc_report.pkl', 'rb'))

In [19]:
performance_data = [[svc_report[key] for key in svc_report], [mnb_report[key] for key in mnb_report],
                   [lr_report[key] for key in lr_report]]
final_report = pd.DataFrame(performance_data, columns = ['test_avg', 'test_std', 'overfit_avg', 'model'])

In [20]:
final_report

,test_avg,test_std,overfit_avg,model
0,0.848043,0.004014,0.146631,SVC
1,0.823479,0.013997,0.107717,MultinomialNB
2,0.842065,0.004337,0.091142,LogisticRegression


In [21]:
# paticularmente, optaria pela regressão logística, foi a segunda métrica mais alta, entretanto, não ficou muito
# abaixo da primeira, apresentou o overfit mais baixo, este, por sua vez, é significativamente mais baixo que
# os demais, já seu desvio padrão, se comporta de forma semelhante à métrica de teste, é a segunda mais alta,
# mas não fica muito abaixo da primeira colocada (neste caso, quanto menor melhor)

In [22]:
# partimos para o pré processamento da base inteira, a fim de treinar um modelo para avaliação, á partir da
# base escondida

# vetorizando variáveis explicativas
# vect = TfidfVectorizer()
# X = vect.fit_transform(df['review']) 

# ajustando regressão logística
# lr = LogisticRegression()
# lr.fit(X, df['label'])

# salvando modelo
# pickle.dump(lr, open('fnl_mdl.pkl', 'wb'))

# FIM